In [2]:
import pandas as pd
import numpy as np


In [ ]:
# Data

df = pd.read_excel("../data/ltcm_exhibits_data.xlsx", sheet_name= 'Exhibit 2', skiprows=2)
df = df.dropna(how='any')
df = df.set_index('Unnamed: 0')
df.index = pd.to_datetime(df.index)
df.index.name = 'Date'
df.head()


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01,1.1,-0.011,-0.013,0.99
1994-04-01,1.1,0.014,0.008,1.00
1994-05-01,1.2,0.068,0.053,1.05
1994-06-01,1.2,-0.039,-0.029,1.02
1994-07-01,1.4,0.116,0.084,1.10


In [11]:
spy_df = pd.read_excel("../data/spy_data.xlsx", sheet_name= 'excess returns')
spy_df = spy_df.dropna(how='any')
spy_df = spy_df.set_index('date')
spy_df.index = pd.to_datetime(spy_df.index)
spy_df.index.name = 'Date'
spy_df.head()

,SPY
Date,
1994-01-31,0.032401
1994-02-28,-0.031964
1994-03-31,-0.050288
1994-04-30,0.007996
1994-05-31,0.012464


In [ ]:
# 2.1

ltcm_gross = df["Gross Monthly Performancea"]  
ltcm_net   = df["Net Monthly Performanceb"] 
spy_excess = spy_df["SPY"]

In [14]:
import statsmodels.api as sm

##　Ｅｘｃｅｒｃｉｃｅ

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel('../data/spx_data_weekly.xlsx',sheet_name="spx data", header=[0,1], index_col=0)
df.index = pd.to_datetime(df.index)
valid_tickers = []
min_weeks = 52 * 5
for ticker in df.columns.levels[0]:
    if ticker not in df.columns: 
        continue
    if df[ticker].dropna().shape[0] >= min_weeks:
        valid_tickers.append(ticker)

df_filtered = df[valid_tickers]
df.head(), df_filtered.shape

(                  A                              AAPL                  \
             PX_LAST EQY_DVD_YLD_IND PE_RATIO  PX_LAST EQY_DVD_YLD_IND   
 date                                                                    
 2015-07-03  36.4044          1.0988  27.4720  28.2589          1.8401   
 2015-07-10  36.2388          1.1038  27.3471  27.5527          1.8873   
 2015-07-17  36.7447          1.0886  27.7288  28.9696          1.7950   
 2015-07-24  36.1560          1.1063  27.2846  27.8253          1.8688   
 2015-07-31  37.6645          1.0620  26.6192  27.1101          1.9181   
 
                         ABBV                             ABNB  ...      YUM  \
            PE_RATIO  PX_LAST EQY_DVD_YLD_IND PE_RATIO PX_LAST  ... PE_RATIO   
 date                                                           ...            
 2015-07-03  13.0592  44.2648          4.6086  11.9239     NaN  ...  18.6689   
 2015-07-10  12.7328  44.9267          4.5407  12.1022     NaN  ...  18.5282   
 2015-

In [6]:
# 1.1
def get_dividend_yield_extremes(df):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1)
    max_yield = div_yld.idxmax(axis=1)  
    min_yield = div_yld.idxmin(axis=1) 
    return pd.DataFrame({'max_yield_ticker': max_yield, 'min_yield_ticker': min_yield})

extremes_df = get_dividend_yield_extremes(df_filtered)
print(extremes_df.tail())

           max_yield_ticker min_yield_ticker
date                                        
2025-05-30              DOW             NVDA
2025-06-06              DOW             NVDA
2025-06-13              DOW             NVDA
2025-06-20              DOW             NVDA
2025-06-27              DOW             NVDA


In [10]:
def get_average_dividend_yield(df, n_weeks=52):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1).tail(n_weeks)
    mean_yield = div_yld.mean()
    max_ticker = mean_yield.idxmax()
    min_ticker = mean_yield.idxmin()
    return max_ticker, mean_yield[max_ticker], min_ticker, mean_yield[min_ticker]

high, high_val, low, low_val = get_average_dividend_yield(df_filtered)
print(f"Highest average dividend ratio：{high}（{high_val:.2f}%）")
print(f"Lowest average dividend ratio：{low}（{low_val:.2f}%）")

Highest average dividend ratio：MO（7.95%）
Lowest average dividend ratio：NVDA（0.03%）


<span style="color: blue;">

ANS: 

Over the past year, MO had the highest average dividend yield at 7.95%, while NVDA had the lowest at just 0.03%.

The elevated yield for MO was primarily due to its consistently high dividend payouts and relatively stable or modestly declining stock price, indicating a strong commitment to income-focused investors. In contrast, NVDA's extremely low yield reflects its minimal dividend distribution and rapidly rising stock price, characteristic of a high-growth company that reinvests earnings rather than returning capital to shareholders.

Overall, the variation in yields appears to be driven more by differences in dividend policy (D) than by changes in price (P), especially in the case of NVDA.

In [12]:
# 1.2

# --- Extract dividend yield and price ---
dvd_yld = df_filtered.xs("EQY_DVD_YLD_IND", axis=1, level=1)
price = df_filtered.xs("PX_LAST", axis=1, level=1)

# --- Weekly returns ---
ret = price.pct_change().shift(-1)

# --- Ranking each week ---
rank = dvd_yld.rank(axis=1, ascending=False)

n = rank.shape[1]
top_n = int(np.floor(0.2 * n))

# --- Long-only weights (equal 0.01) ---
weights_long = (rank <= top_n).astype(float) * 0.01

# --- Portfolio return ---
carry_ret = (weights_long * ret).sum(axis=1)
carry_ret = carry_ret.dropna()
carry_ret.head()




date
2015-07-03    0.005380
2015-07-10    0.004836
2015-07-17   -0.018145
2015-07-24    0.017742
2015-07-31   -0.005158
dtype: float64

In [ ]:
# 1.3
# --- Long-Short weights ---
weights_ls = pd.DataFrame(0, index=rank.index, columns=rank.columns)

weights_ls[rank <= top_n] = 0.01
weights_ls[rank > (n - top_n)] = -0.01

# --- Long-short portfolio return ---
ls_ret = (weights_ls * ret).sum(axis=1)
ls_ret = ls_ret.dropna()
ls_ret.head()

C:\Users\User\AppData\Local\Temp\ipykernel_18224\3384924985.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank <= top_n] = 0.01
C:\Users\User\AppData\Local\Temp\ipykernel_18224\3384924985.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank > (n - top_n)] = -0.01


date
2015-07-03    0.005102
2015-07-10    0.000766
2015-07-17   -0.009259
2015-07-24    0.009111
2015-07-31   -0.002786
dtype: float64

In [15]:
# 1.4

def perf_stats(r):
    ann_mean = r.mean() * 52
    ann_vol = r.std() * np.sqrt(52)
    sharpe = ann_mean / ann_vol
    
    skew = r.skew()
    var_5 = r.quantile(0.05)
    cvar_5 = r[r <= var_5].mean()
    
    cum = (1 + r).cumprod()
    dd = cum / cum.cummax() - 1
    max_dd = dd.min()
    
    return pd.Series({
        "Mean": ann_mean,
        "Vol": ann_vol,
        "Sharpe": sharpe,
        "Skew": skew,
        "VaR_5%": var_5,
        "CVaR_5%": cvar_5,
        "MaxDD": max_dd
    })

df_add = pd.read_excel(
    '../data/spx_data_weekly.xlsx',
    sheet_name='additional data',
    header=[0,1],
    index_col=0
)

df_add.index = pd.to_datetime(df_add.index)
spy_price = df_add.xs("SPY", axis=1, level=0).xs("PX_LAST", axis=1)
spy_ret = spy_price.pct_change().shift(-1).dropna()
common_idx = carry_ret.index.intersection(spy_ret.index)

carry_ret = carry_ret.loc[common_idx]
ls_ret = ls_ret.loc[common_idx]
spy_ret = spy_ret.loc[common_idx]

perf_table = pd.concat([
    perf_stats(carry_ret),
    perf_stats(ls_ret),
    perf_stats(spy_ret)
], axis=1)

perf_table.columns = ["Carry Long", "Carry Long-Short", "SPY"]
print(perf_table)

         Carry Long  Carry Long-Short       SPY
Mean       0.112670          0.021356  0.140826
Vol        0.160823          0.095895  0.173193
Sharpe     0.700582          0.222706  0.813116
Skew       0.285957          1.559453 -0.595027
VaR_5%    -0.028205         -0.018282 -0.033571
CVaR_5%   -0.049354         -0.026737 -0.056766
MaxDD     -0.347861         -0.179768 -0.318291
